In [1]:
import numpy as np
import pandas as pd

In [2]:
data=pd.read_table("NLPlabeledData.tsv")

In [3]:
from nltk.corpus import stopwords
import nltk

In [4]:
nltk.download("stopwords")
list=stopwords.words("english")
list1=stopwords.words("turkish")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ipek\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
ex_text=data.review[0]

In [6]:
len(ex_text)

2302

In [7]:
from bs4 import BeautifulSoup

In [8]:
ex_text1=BeautifulSoup(ex_text, features="lxml").get_text() #to clean from html tags

In [11]:
import re
ex_text2=re.sub("[^a-zA-Z]"," ",ex_text1)

In [12]:
ex_text3=ex_text2.lower()
ex_text3=ex_text3.split()

In [13]:
len(ex_text3)

437

In [17]:
swords=set(stopwords.words("english"))
ex_text4=[w for w in ex_text3 if w not in swords]

In [21]:
def process(review):
    review=BeautifulSoup(review, features="lxml").get_text()
    review=re.sub("[^a-zA-Z]"," ",review)
    review=review.lower()
    review=review.split()
    review=[w for w in review if w not in swords]   #we remove the stop words
    return(" ".join(review))   #will add a space at the beginning and send back every value we send, the characters we split with the split will be listed again.

In [23]:
train_X_all=[]
for i in range(len(data["review"])):
    if(i+1)%1000==0:
        print("process is completed",i+1)
    train_X_all.append(process(data["review"][i]))  #review is a column name

C:\Users\ipek\AppData\Local\Temp\ipykernel_7344\2022276758.py:2: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  review=BeautifulSoup(review, features="lxml").get_text()


process is completed 1000
process is completed 2000
process is completed 3000
process is completed 4000
process is completed 5000
process is completed 6000
process is completed 7000
process is completed 8000
process is completed 9000
process is completed 10000
process is completed 11000
process is completed 12000
process is completed 13000
process is completed 14000
process is completed 15000
process is completed 16000
process is completed 17000
process is completed 18000
process is completed 19000
process is completed 20000
process is completed 21000
process is completed 22000
process is completed 23000
process is completed 24000
process is completed 25000


In [25]:
#you can use another library
train_X_all1=[]
from tqdm import tqdm
for i in tqdm(range(len(data["review"]))):
    train_X_all1.append(process(data["review"][i]))

  0%|          | 0/25000 [00:00<?, ?it/s]C:\Users\ipek\AppData\Local\Temp\ipykernel_7344\2022276758.py:2: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  review=BeautifulSoup(review, features="lxml").get_text()
100%|██████████| 25000/25000 [00:05<00:00, 4197.32it/s]


In [ ]:
train_X_all

In [33]:
from sklearn.model_selection import train_test_split
X=train_X_all
y=np.array(data["sentiment"])
Xtrain,Xtest,ytrain,ytest=train_test_split(X,y,test_size=0.2,random_state=42)

In [34]:
from sklearn.feature_extraction.text import CountVectorizer
Vectorizer = CountVectorizer(max_features=5000)   #set it to only take the 5000 most common words, if the words occur in sentences, it takes the values 1 if not, it takes the values 0

In [35]:
Xtrain=Vectorizer.fit_transform(Xtrain)
Xtrain=Xtrain.toarray()

In [36]:
#naive bayes algorithm
from sklearn.naive_bayes import GaussianNB
nb=GaussianNB()
nb.fit(Xtrain,ytrain)
Xtest=Vectorizer.transform(Xtest)

In [37]:
Xtest=Xtest.toarray()

In [47]:
from sklearn.metrics import roc_auc_score
ytest_pred=nb.predict(Xtest)
accuracy=roc_auc_score(ytest,ytest_pred)
print("accuracy: ",accuracy)

accuracy:  0.7427564216109123


In [52]:
# #random forest algorithm
# from sklearn.ensemble import RandomForestClassifier
# rf=RandomForestClassifier(n_estimators=100, random_state=42) burası eksik random forest naive bayese göre daha iyi bir sonuç veriyor, tamamla!!!

In [55]:
positive_sent = "The innovative AI model demonstrated exceptional performance, surpassing expectations in its ability to analyze and interpret vast amounts of data with unprecedented speed and accuracy, proving to be a game-changer in the field of machine learning"
negative_sent = "Despite the high hopes placed on the advanced software, it ultimately failed to deliver on its promises, struggling with the interpretation of nuanced and complex datasets, leading to significant inaccuracies and undermining its reliability in critical applications"

In [57]:
# Preprocessing new sentences
positive_processed = process(positive_sent)
negative_processed = process(negative_sent)

# Vectorizing the new sentences
positive_vectorized = Vectorizer.transform([positive_processed]).toarray()
negative_vectorized = Vectorizer.transform([negative_processed]).toarray()

# Making predictions
positive_prediction = nb.predict(positive_vectorized)
negative_prediction = nb.predict(negative_vectorized)

# Outputting the results
print("Positive Sentence Prediction:", "Positive" if positive_prediction[0] == 1 else "Negative")
print("Negative Sentence Prediction:", "Positive" if negative_prediction[0] == 1 else "Negative")


Positive Sentence Prediction: Negative
Negative Sentence Prediction: Negative


In [59]:
olumlu="The natural language processing system exhibits impressive performance in text-based tasks, showcasing its advanced capabilities in understanding and processing human language"
olumsuz="Unfortunately, the natural language processing model struggles with intricate nuances, leading to a compromised performance in various language-based applications"
manuel_olumlu=process(olumlu)
manuel_olumlu=Vectorizer.transform([manuel_olumlu])
manuel_olumlu=manuel_olumlu.toarray()
sonuc1=nb.predict(manuel_olumlu)
print(sonuc1)
if sonuc1==1:
    print("positive")
else:
    print("negative")

[0]
olumsuz
